In [ ]:
import pandas as pd

crime_raw_data = pd.read_csv('./data/2023년.csv', encoding='euc-kr')

crime_raw_data['죄종'] = crime_raw_data['죄종'].replace('강간,추행', '강간')

crime_station = crime_raw_data.pivot_table(
    index="구분", columns=["죄종", "발생검거"], aggfunc="sum"
)

crime_station.columns = crime_station.columns.droplevel([0])

tmp = crime_station.columns.get_level_values(0) + \
    crime_station.columns.get_level_values(1)

crime_station.columns = tmp

crime_station.rename(
    columns={
        "강간검거": "성범죄검거",
        "강간발생": "성범죄발생",
    },
    inplace=True,
)

real_gu = {
    "종로구": ["종로", "혜화"],
    "중구": ["남대문", "중부"],
    "용산구": ["용산"],
    "성동구": ["성동"],
    "광진구": ["광진"],
    "동대문구": ["동대문"],
    "중랑구": ["중랑"],
    "성북구": ["성북", "종암"],
    "강북구": ["강북"],
    "도봉구": ["도봉"],
    "노원구": ["노원"],
    "은평구": ["서부", "은평"],
    "서대문구": ["서대문"],
    "마포구": ["마포"],
    "양천구": ["양천"],
    "강서구": ["강서"],
    "구로구": ["구로"],
    "금천구": ["금천"],
    "영등포구": ["영등포"],
    "동작구": ["동작"],
    "관악구": ["관악"],
    "서초구": ["방배", "서초"],
    "강남구": ["강남", "수서"],
    "송파구": ["송파"],
    "강동구": ["강동"],
}

tmp_gu = []

for station in crime_station.index:
    for gu_name, station_name in real_gu.items():
        if station in station_name:
            tmp_gu.append(gu_name)

crime_station["구"] = tmp_gu

crime_gu = crime_station.pivot_table(index="구", aggfunc="sum")


crime_gu

,강도검거,강도발생,살인검거,살인발생,성범죄검거,성범죄발생,절도검거,절도발생,폭력검거,폭력발생
구,,,,,,,,,,
강남구,18,20,11,14,567,678,1284,2480,3112,3571
강동구,4,5,6,4,122,144,980,1676,1383,1569
강북구,1,1,7,7,102,141,730,917,1302,1431
강서구,4,6,2,1,225,300,1243,1794,2070,2195
관악구,10,10,17,16,268,328,1166,2013,2035,2402
광진구,3,5,2,2,178,238,830,1616,1320,1563
구로구,7,7,12,12,144,194,849,1624,1620,1900
금천구,4,5,3,4,95,128,629,987,1022,1116
노원구,4,3,6,7,131,159,845,1632,1621,1766


In [43]:
# 검거 칼럼들만 합산
sum_arrest = crime_gu[['강도검거', '살인검거', '성범죄검거', '절도검거', '폭력검거']].sum(axis=1)

# 발생 칼럼들만 합산
sum_occurrence = crime_gu[['강도발생', '살인발생', '성범죄발생', '절도발생', '폭력발생']].sum(axis=1)

# 발생 합계를 정규화
crime_gu['정규화'] = sum_occurrence / sum_occurrence.max()

# 새로운 칼럼에 합산값 할당
crime_gu['검거 합계'] = sum_arrest
crime_gu['발생 합계'] = sum_occurrence
crime_gu['검거율'] = (crime_gu['검거 합계'] / crime_gu['발생 합계']) * 100

# 검거율 소수점 첫째 자리까지 반올림
crime_gu['검거율'] = crime_gu['검거율'].round(1)

# 결과 확인
crime_gu[['검거 합계', '발생 합계', '검거율','정규화']]

,검거 합계,발생 합계,검거율,정규화
구,,,,
강남구,4992,6763,73.8,1.000000
강동구,2495,3398,73.4,0.502440
강북구,2142,2497,85.8,0.369215
강서구,3544,4296,82.5,0.635221
관악구,3496,4769,73.3,0.705160
광진구,2333,3424,68.1,0.506284
구로구,2632,3737,70.4,0.552565
금천구,1753,2240,78.3,0.331214
노원구,2607,3567,73.1,0.527429
